# Results Prototype Notebook

## Framing

This notebook generates the linear probing and inter-class distance metrics for a trained Perceiver Autoencoder model (with or without predictive loss). 

## Design
 - Load the `validate` set from Kinetics 400. 
     - Parse the text file in `Multimodal_Foundation/datasets/Kinetics/frame_cnts_val.txt` to get the actual number of frames. 
     - Load some random segment from that number of frames (use random seed or store this somehow). 
     - Figure out how to make a `tf.Dataset` with labels from this. 
 - Load a trained model, parse the data, generate latent representations of each video in the validation set. 
 - Do a cheeky PCA on the data, take a look. 
 - Figure out how the ViT-MAE paper performed linear probing. 
 - Perform linear probing, compute top-5 accuracy. 

In [1]:
## Import box
import os 
import sys 
import random
import pathlib
import itertools
import collections
import math
import argparse
import datetime
import pickle
import pdb
import time
from packaging import version

os.environ["TF_GPU_THREAD_MODE"] = "gpu_private"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

sys.path.append("../src")       # Adding src folder to path
sys.path.append("../src/utils") # Adding utils folder to path

import tensorflow as tf 
from tensorflow import keras
# import tensorflow_addons as tfa
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
import imageio

import model
import video_loader as vl
import video_preprocess as vp
import train_m2
import parallel_video_loader as pvl

2022-12-21 10:00:01.491344: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-21 10:00:01.633127: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-21 10:00:01.661182: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-21 10:00:02.293063: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

LOGICAL GPUs:  [LogicalDevice(name='/device:GPU:0', device_type='GPU'), LogicalDevice(name='/device:GPU:1', device_type='GPU')]


2022-12-21 10:00:03.308089: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-21 10:00:04.750819: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2022-12-21 10:00:04.750861: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21403 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090 Ti, pci bus id: 0000:1a:00.0, compute capability: 8.6
2022-12-21 10:00:04.751739: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWT

In [2]:
## Constants
EXPERIMENT_FOLDER = ... 

## 0: Load Experiment Params

To make sure we format the data properly, we need to know the contents of `data_format_args.pkl` from the relevant experiment. Let's load this dict here:

In [3]:
# Load `data_format_args.pkl` !

## 1: Loading the Data (Kinetics-400 Validation Set)

The relative path to the Kinetics 400 validation set is `../datasets/Kinetics/validate`. Within this folder, each of the 400 action classes has a subfolder containing videos of that class. 

There is also a text file called `frame_cnts_val.txt`. Each line pertains to one video file and is of the form `Easy: 411.0, hard: 301 -- validate/testifying/_yMeY9-Gi0k.mp4`. "Easy" refers to the number of frames as counted by the OpenCV default frame counting system, whereas "hard" refers to the number of frames as counted when actually reading in the videos. After that is the path of the video. The "hard" count for the number of frames should be used when sampling frames from the video as the "easy" count generally overestimates. 

In [4]:
frame_count_file_path = "../datasets/Kinetics/validate/frame_cnts_val.txt"
dataset_path = "../datasets/Kinetics"

### 1a: Load `frame_cnts_val.txt`

This should be loaded as a list of tuples of `(path, frame_count)`. 

In [5]:
def get_frame_counts(path_to_counts): 
    ... 

frame_counts = get_frame_counts(frame_count_file_path)

### 1b: Load a List of Video Matrices (`numpy`)

Use `parallel_video_loader`'s (`pvl`'s) `load_video_range(video_path, start_frame, num_frames, output_size)` function to load a random subset of the frames. For now, we can use the following parameters: 
 - `num_frames`: 4
 - `output_size`: (120,180)
 - `start_frame`: 0 

Use the `frame_counts` tuple to get the paths of each video and to make sure they have sufficiently many frames.


In [6]:
... 

Ellipsis

### 1c: Convert/Process the List of Videos $\to$ `tf.Tensor`

Follow the procedure in [vp.get_videosets()](https://github.com/amanb2000/Multimodal_Foundation/blob/06bacc67f37c6dfb9de811ca78345fa312d9964d/src/utils/video_preprocess.py#L238) to convert the list of numpy matrices into patched/positionally encoded tensors. 

Optimally, we still want this to be a regular python `list` rather than a Tensorflow `Dataset`. This will make it easier to keep track of the labels in a parallel list. 

In [7]:
...

Ellipsis

## 2: Load the Model 

From `EXPERIMENT_FOLDER` we need to instantiate and load the model weights. We can follow the procedure from [execute_experiment.py](https://github.com/amanb2000/Multimodal_Foundation/blob/06bacc67f37c6dfb9de811ca78345fa312d9964d/src/execute_experiment.py#L445)'s code that does this. 

In [8]:
...

Ellipsis

## 3: Create Embeddings using the Model

`NOTE`: We need to make sure our positional codes match the ones the model is expecting. Be sure to generate them, following `execute_experiment.py`'s method of doing that, and append the correct ones to the `FlatPatchSet`-equivalent Tensors. 

Store the embeddings in another parallel list. 

## 4: Some Cheeky PCA

Also do the inter-cluster distances here :) 

## 5: Linear Probing (Simple sklearn)

Once done, make sure this is actually what they did in ViT-MAE...